## Fake News Classificer Using Bidirectional LSTM

In [1]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.shape

(20800, 5)

In [6]:
df=df.dropna()

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## get the independent features
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
## Check whether dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.15.0'

In [14]:
#importing necessary libraries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [15]:
## vocabulary size
voc_size=10000

In [16]:
news=X.copy()

In [17]:
news.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [18]:
#Importing NLP Libraries
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
news.reset_index(inplace=True)

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(news)):
    review = re.sub('[^a-zA-Z]', ' ', news['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[8428, 958, 5421, 1117, 2987, 91, 8231, 9034, 2178, 9828],
 [5, 4441, 2027, 5881, 5846, 46, 7192],
 [9377, 8831, 378, 1787],
 [2090, 5953, 3679, 4085, 4412, 1392],
 [5809, 5846, 8573, 1108, 4194, 7759, 5846, 9954, 8429, 7336],
 [2088,
  2368,
  9393,
  3765,
  4871,
  9363,
  8765,
  7842,
  8634,
  211,
  7501,
  1293,
  5580,
  9955,
  7192],
 [2707, 1963, 7904, 738, 9979, 5671, 1444, 5930, 4807, 6445, 749],
 [5725, 3200, 9033, 4940, 6348, 4957, 9363, 6756, 4807, 6445, 749],
 [5013, 6511, 637, 9480, 3136, 4511, 9414, 4530, 9363, 4668],
 [5456, 4608, 4363, 4338, 9991, 5466, 4842, 2259],
 [3555, 9861, 9618, 2657, 9074, 2341, 9776, 1109, 3589, 3356, 9337],
 [4085, 5703, 2987, 4511, 9363, 6348],
 [5477, 6491, 4868, 5111, 2427, 9842, 3904, 1695, 8796],
 [9582, 9847, 4913, 4112, 4322, 7174, 3715, 4807, 6445, 749],
 [3826, 3286, 7813, 5084, 7364, 4807, 6445, 749],
 [7404, 4240, 7906, 5006, 1753, 6896, 9053, 2985, 2688, 4192],
 [6855, 3009, 4441],
 [6554, 6576, 7497, 2767, 9363, 8812, 910, 

## Embedding Representation

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 9034, 2178, 9828],
       [   0,    0,    0, ..., 5846,   46, 7192],
       [   0,    0,    0, ..., 8831,  378, 1787],
       ...,
       [   0,    0,    0, ..., 4807, 6445,  749],
       [   0,    0,    0, ..., 1062, 8503, 1349],
       [   0,    0,    0, ..., 3068, 2735, 2891]], dtype=int32)

In [25]:
## Creating model
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            500000    
                                                                 
 bidirectional (Bidirection  (None, 300)               241200    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 741501 (2.83 MB)
Trainable params: 741501 (2.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [28]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=32)

Epoch 1/12
383/383 [==============================] - 23s 44ms/step - loss: 0.2481 - accuracy: 0.8946 - val_loss: 0.1931 - val_accuracy: 0.9238
Epoch 2/12
383/383 [==============================] - 4s 11ms/step - loss: 0.1133 - accuracy: 0.9567 - val_loss: 0.2064 - val_accuracy: 0.9231
Epoch 3/12
383/383 [==============================] - 4s 10ms/step - loss: 0.0620 - accuracy: 0.9801 - val_loss: 0.3239 - val_accuracy: 0.9167
Epoch 4/12
383/383 [==============================] - 5s 12ms/step - loss: 0.0400 - accuracy: 0.9867 - val_loss: 0.3582 - val_accuracy: 0.9133
Epoch 5/12
383/383 [==============================] - 3s 9ms/step - loss: 0.0202 - accuracy: 0.9936 - val_loss: 0.3794 - val_accuracy: 0.9153
Epoch 6/12
383/383 [==============================] - 3s 9ms/step - loss: 0.0115 - accuracy: 0.9967 - val_loss: 0.4961 - val_accuracy: 0.9140
Epoch 7/12
383/383 [==============================] - 4s 11ms/step - loss: 0.0077 - accuracy: 0.9976 - val_loss: 0.5666 - val_accuracy: 0.9117


In [29]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [30]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [31]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3129,  290],
       [ 242, 2374]])

In [32]:
print(accuracy_score(y_test,y_pred))

0.911847555923778


In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

